In [20]:
import sys, importlib, pandas as pd, os
import meta_test
importlib.reload(meta_test)

reproduccion = ['--destruct 120 --temp_ini 980 --alpha 0.997 --prob_CambiarPrimarios 0.15 --prob_CambiarSecundarios 0.15 --prob_MoverPaciente_bloque 0.20 --prob_MoverPaciente_dia 0.10 --prob_EliminarPaciente 0.10 --prob_AgregarPaciente_1 0.24 --prob_AgregarPaciente_2 0.24 --prob_DestruirAgregar10 0.01 --prob_DestruirAfinidad_Todos 0.30 --prob_DestruirAfinidad_Uno 0.30 --prob_PeorOR 0.35 --prob_AniquilarAfinidad 0.05 --prob_MejorarAfinidad_primario 0.20 --prob_MejorarAfinidad_secundario 0.20 --prob_AdelantarDia 0.10 --prob_MejorOR 0.15 --prob_AdelantarTodos 0.05 --prob_CambiarPaciente1 0.20 --prob_CambiarPaciente2 0.20 --prob_CambiarPaciente3 0.20 --prob_CambiarPaciente4 0.20 --prob_CambiarPaciente5 0.20 --destruct_type 1 --prob_DestruirOR 0.00 --prob_elite 0.3 --prob_GRASP 0.90 --prob_normal 0.10 --prob_Busq 0.99 --BusqTemp yes --GRASP_alpha 0.25 --elite_size 4 --prob_GRASP1 0.50 --prob_GRASP2 0.30 --prob_GRASP3 0.20 --acceptance_criterion SA']
NUM_FLAGS = [
    "--destruct","--temp_ini","--alpha",
    "--prob_CambiarPrimarios","--prob_CambiarSecundarios",
    "--prob_MoverPaciente_bloque","--prob_MoverPaciente_dia",
    "--prob_EliminarPaciente","--prob_AgregarPaciente_1","--prob_AgregarPaciente_2",
    "--prob_DestruirAgregar10","--prob_DestruirAfinidad_Todos","--prob_DestruirAfinidad_Uno",
    "--prob_PeorOR", "--prob_AniquilarAfinidad",
    "--prob_MejorarAfinidad_primario","--prob_MejorarAfinidad_secundario",
    "--prob_AdelantarDia","--prob_MejorOR","--prob_AdelantarTodos",
    "--prob_CambiarPaciente1","--prob_CambiarPaciente2","--prob_CambiarPaciente3",
    "--prob_CambiarPaciente4","--prob_CambiarPaciente5",
    "--destruct_type","--prob_DestruirOR","--prob_elite","--prob_GRASP","--prob_normal",
    "--prob_Busq","--GRASP_alpha","--elite_size",
    "--prob_GRASP1","--prob_GRASP2","--prob_GRASP3"
]
CAT_FLAGS = ["--BusqTemp","--acceptance_criterion"]
ALL_FLAGS = NUM_FLAGS + CAT_FLAGS

DEFAULTS = {f:0 for f in NUM_FLAGS}
DEFAULTS.update({
    "--BusqTemp":"yes",
    "--acceptance_criterion":"SA"})

def parse_line(line:str)->dict:
    toks = line.split()
    return {toks[i]:toks[i+1] for i in range(0,len(toks),2)}

parsed = [{**DEFAULTS, **parse_line(l)} for l in reproduccion]
df = pd.DataFrame(parsed)
df[NUM_FLAGS] = df[NUM_FLAGS].apply(pd.to_numeric, errors='coerce').fillna(0)

GROUP_I = [c for c in NUM_FLAGS if any(k in c for k in
    ["CambiarPrimarios","CambiarSecundarios","MoverPaciente","EliminarPaciente",
     "AgregarPaciente","DestruirAgregar10","DestruirAfinidad","PeorOR", "AniquilarAfinidad"])]
GROUP_II = [c for c in NUM_FLAGS if any(k in c for k in
    ["MejorarAfinidad","AdelantarDia","MejorOR","AdelantarTodos","CambiarPaciente"])]
GROUP_III = ["--prob_DestruirOR","--prob_elite","--prob_GRASP","--prob_normal"]
GROUP_IV = ["--prob_GRASP1","--prob_GRASP2","--prob_GRASP3"]

def _norm(cols):
    vals   = df[cols].values
    totals = vals.sum(axis=1, keepdims=True)
    totals[totals == 0] = 1
    df[cols] = vals / totals

for group in (GROUP_I, GROUP_II, GROUP_III, GROUP_IV):
    _norm(group)
df_out = (
    df[ALL_FLAGS]
      .T
      .reset_index()
      .rename(columns={"index": "Parameter"})
)

METRIC_NAMES = ["Promedio","Mejor","Promedio_Gap","Mejor_Gap",
                "Tiempo","AvgIter","BestIter","NumSched"]

n_exec     = len(reproduccion)
cols_lvl1  = [f"Ejec{j+1}" for j in range(n_exec) for _ in METRIC_NAMES]
cols_lvl2  = METRIC_NAMES * n_exec
multi_cols = pd.MultiIndex.from_arrays([cols_lvl1, cols_lvl2])

df_gaps = pd.DataFrame(columns=multi_cols)
df_gaps.insert(0, ("", ""), []) 


def flush_to_excel(df_gaps_partial: pd.DataFrame) -> None:
    path   = "reproduccion.xlsx"
    exists = os.path.exists(path)
    mode   = "a" if exists else "w"

    writer_kwargs = {
      "engine": "openpyxl",
      "mode":   mode
    }
    if exists:
        writer_kwargs["if_sheet_exists"] = "replace"

    with pd.ExcelWriter(path, **writer_kwargs) as w:
        df_out.to_excel(w, sheet_name="Parámetros", float_format="%.4f")
        df_gaps_partial.to_excel(w, sheet_name="Gaps",       float_format="%.4f")

flush_to_excel(df_gaps)

for inst in range(1, 6):
    df_gaps.loc[inst-1, ("", "")] = f"Instancia {inst}"
    for conf_idx, line in enumerate(reproduccion, start=1):
        argv = ["meta_test.py", "0", "0", "0", f"../irace/instances/instance{inst}.json"] + line.split()
        sys.argv = argv
        print(f"Instancia {inst:02d}  Ejec. {conf_idx:02d}")
        result = meta_test.main()
        start = (conf_idx - 1) * len(METRIC_NAMES)
        stop  = start + len(METRIC_NAMES)
        df_gaps.iloc[inst-1, 1+start : 1+stop] = result
        flush_to_excel(df_gaps)

print("✅ Todo terminado; resultados guardados en 'reproduccion.xlsx'")

Instancia 01  Ejec. 01
-88142.0
Instancia 02  Ejec. 01
-96112.66666666667
Instancia 03  Ejec. 01


KeyboardInterrupt: 